In [77]:
import numpy as np
import pandas as pd

In [54]:
df = pd.read_csv("../data/Checking.csv", encoding='iso-8859-1')
df.head()

,Date,nps,Comment
0,4/17/2023 20:23,9,PRECIO
1,4/17/2023 20:21,9,ES LA PRIMERA VEZ Y SI ME SIENTO CON ALGO DE I...
2,4/17/2023 20:17,9,Es muy rapido
3,4/17/2023 20:17,10,Bueno
4,4/17/2023 20:10,9,Prcticidad


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17860 entries, 0 to 17859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     17860 non-null  object
 1   nps      17860 non-null  int64 
 2   Comment  17835 non-null  object
dtypes: int64(1), object(2)
memory usage: 418.7+ KB


In [88]:
labels_column = df.nps#.apply(lambda x: "B" if x > 8 else "lol")
bins = [6, 7, 8, 10]
labels = pd.cut(labels_column, bins=bins, labels=["M", "R", "B"])

In [89]:
labels.value_counts()

B    16973
R      339
M      138
Name: nps, dtype: int64

In [57]:
sents = df.Comment[30:70].to_list()
mini_checking = pd.DataFrame({"Comment": sents})
print(sents)

['es muy facil', 'ágil', 'rapidos', 'POR LA RAPIDEZ', 'super facil', 'ES MUY FACIL', 'si', 'rapida y eficiente la página', 'Muy eficiente', 'Es muy fácil y rápido.', 'excelente', 'Es muy sencillo y rápido todo, gracias', 'Facil de usar', 'conveniente y rapido', 'por la facilidad', 'servicio en webb', 'POR LA FACILIDAD DEL TRAMITE', 'Amigable', 'FACILIDAD AL HACERLO, ME EVITO TRAMITES EN EL AEROPUERTO', 'La flexibilidad.', 'buen servicio', 'ES CON LOS QUE YO VUELO', 'todo excelente', 'Facil acceso', 'Sencillez', 'EL SERVICIO ES RÁPIDO', 'MUY FACIL', 'RAPIDEZ Y PRACTICIDAD', 'por la rapidez', 'muy buen servicio', 'Buena y fácil de usar', 'Muy facil', 'Todo el proceso fue rápido y sencillo.', 'POR SU RAPIDEZ', 'TODO MUY BIEN', 'excellent servicio', 'La página web es muy práctica', 'muy birn', 'claro', 'AGIL Y FACIL PROCESO']


In [58]:
sents = ['too easy', 'agile', 'rapid', 'FOR THE RAPIDITY', 'super facil', 'TOO EASY', 'and', 'fast and efficient page', 'Very efficient',
 'It is very easy and fast.',  'great',  'Everything is very simple and fast, thank you',  'Easy to use',  'convenient and fast',
 'by the ease',  'service on webb',  'BECAUSE OF THE EASE OF THE PROCESS',  'Friendly',  'EASILY DOING IT, I AVOID PROCEDURES AT THE AIRPORT',
 'Flexibility.',  'good service',  'IT IS WITH WHOM I FLY',  'all excellent',  'Easy access',  'Simplicity',  'THE SERVICE IS FAST',
 'VERY EASY',  'SPEED AND PRACTICALITY',  'for the speed', 'very good service', 'good and easy to use', 'Very easy',
  'The whole process was quick and easy.',  'FOR ITS SPEED', 'ALL VERY WELL', 'excellent service', 'The website is very practical',
 'very well', 'Of course', 'AGILE AND EASY PROCESS']

In [59]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [60]:
def custom_standardization(data):
    lower = tf.strings.lower(data)
    return lower

In [61]:
dataset = tf.data.Dataset.from_tensor_slices(sents).batch(32)
# list(dataset.as_numpy_iterator())

In [62]:
vocab_size = 10000
sequence_length = 100

vectorizer = TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [63]:
vectorizer.adapt(sents)

In [64]:
vectorizer.vocabulary_size()

61

In [65]:
vectorizer.get_vocabulary()[5:15]

['and', 'service', 'is', 'fast', 'speed', 'process', 'of', 'it', 'good', 'for']

In [66]:
output = vectorizer([["i like it a lot"]])
output.numpy()[0, :6]

array([19,  1, 12,  1,  1,  0], dtype=int64)

In [67]:
embedding_dim=16

model = Sequential([
  vectorizer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(3)
])

In [68]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [69]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")